In [ ]:
# Copyright 2024 Stephan Bscheider sbsch@bu.edu
# Copyright 2024 Humzah Durrani hhd8@bu.edu
# Copyright 2024 Alex Tianji Sun tianjis@bu.edu

## Settings required for chess.com board

Pieces: Neo</br>
Board: Brown</br>
Coordinates: None</br>
Piece Notation: Figurine</br>
Move Method: Click Squares</br>
Highlight Moves: OFF</br>
Show Legal Moves: OFF</br>

# How to Run
Login to chess.dot, set the board to starting position</br>
Make sure chess window is on your main monitor</br>
Run all cells </br>
make moves by clicking a piece and then clicking on the target square, DO NOT DRAG PIECE WITH MOUSE!!</br>
Hold q to kill the main while loop</br>
Enjoy your moveList.csv</br>
(Not yet properly integrated with chess engine, but chess engine code takes input in the form of the move output from here so its basically there.)


In [51]:
# Importing libraries for every subsequent notebook cell
import cv2 as cv
from PIL import Image, ImageGrab 
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import csv
import keyboard

# Some intializations of variables, this may or may not be needed here 
letters = []
numbers = []
white = True
control_contours = []

def shift_contour(contour, x_offset, y_offset):
    # Casual function for shifting the xy pos of a contour
    shifted_contour = contour.copy()  ## Copy the contour to avoid modifying the original
    # Add the offset to all points in the contour
    shifted_contour[:,:,1] += x_offset  # Shift X coordinates
    shifted_contour[:,:,0] += y_offset  # Shift Y coordinates
    return shifted_contour

def piece_identifier(contour,control_contours):
    # Function for using the cv.matchShapes to compare contours on the real board to the 6 control contours
    similarity_array = []
    
    for piece in control_contours:
        similarity = cv.matchShapes(piece[0], contour, cv.CONTOURS_MATCH_I1, 0.0)
        similarity_array.append(similarity)
    
    # The the lower the value the more similar the contours are so this checks that.
    if similarity_array.index(min(similarity_array)) == 0:
        return "p" #"Pawn"
    elif similarity_array.index(min(similarity_array)) == 1:
        return "n" #"Knight"
    elif similarity_array.index(min(similarity_array)) == 2:
        return "b" #"Bishop"
    elif similarity_array.index(min(similarity_array)) == 3:
        return "r" #"Rook"
    elif similarity_array.index(min(similarity_array)) == 4:
        return "q" #"Queen"
    elif similarity_array.index(min(similarity_array)) == 5:
        return "k" #"King"
    else:
        pass


def board_finder(board_im):
    # Function for finding the actual chess board size and position from the screenshot of the whole screen.
    
    # Open CV is in BGR so this funct converts it to RGB for ease of use
    im_rgb = cv.cvtColor(board_im, cv.COLOR_BGR2RGB)

    # Define the color to match (in RGB)
    target_color = [237, 214, 176]

    # Find all pixels matching the target color
    mask = cv.inRange(im_rgb, np.array(target_color), np.array(target_color))
    matching_pixels = cv.findNonZero(mask)

    xArray = []
    yArray = []
    if matching_pixels is not None:
    
        for pixel in matching_pixels:
            x, y = pixel[0]
            xArray.append(x)
            yArray.append(y)
        
        # Makes the arrays into a set, so that it contains only one of each unique value, then
        # back into a list
        xArray = list(set(xArray))
        yArray = list(set(yArray))
        left = min(xArray)
        right = max(xArray) + 1
        top = min(yArray)
        bottom = max(yArray) + 1

    return left, right, top, bottom

def board_state(board_img_file,square_size):
    # Function to find the countours of the pieces on the board, identify them and their location and then save the board as a 2D array for later use.
    
    im2 = cv.imread(board_img_file)    
    black_pieces_contours = [ ]
    white_pieces_contours = [ ]
    
    # Initialize the board as a 2D array if "." for empty squares to be filled with pieces later
    board = [["." for _ in range(8)] for _ in range(8)]
    
    # Iterate over the 8x8 squares of the board
    for i in range(8):
        for j in range(8):
            
            # Location of the current square based on the sizes of the squares
            x = square_size*i
            y = square_size*j

            square = im2[x:x+square_size, y:y+square_size]
            # Convert the image to grayscaleq
            gray = cv.cvtColor(square, cv.COLOR_BGR2GRAY)
            # Apply GaussianBlur to reduce image noise and improve contour detection
            blurred = cv.GaussianBlur(gray, (3, 3), 0)
            # Use adaptive thresholding to create a binary image
            thresh = cv.adaptiveThreshold(blurred, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY_INV, 7, 1)
            # Find contours in the thresholded image
            contours, _ = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
            
            temp_contours = []
            
            # We were using this min and max countour area earlier on but I dont think this check is necessary anymore but it still works so
            min_contour_area = 50  # Minimum contour area to be considered a piece
            max_contour_area = 10000
            
            for cnt in contours:
                if min_contour_area <  cv.contourArea(cnt) < max_contour_area:
                    temp_contours.append(cnt)
                    # Check the color of the piece in a certain location where there should be a consistent color no matter the shape of a piece
                    piece_color = square[int(0.8*square_size), int(0.5*square_size)]
                    # Us piece_identifier to identify the piece type
                    piece_type = piece_identifier(temp_contours[0],control_contours)
                else:
                    pass
                
            if len(temp_contours) > 0:
                # So if there is a contour on this square it will be added to the correct color pieces list and shifted to the correct location
                if piece_color[0] == 87:
                    shifted_contour = shift_contour(temp_contours[0], x, y)
                    black_pieces_contours.append(shifted_contour)
                    board[i][j] = f"{piece_type}"
                elif piece_color[0] == 249:
                    shifted_contour = shift_contour(temp_contours[0], x, y)
                    white_pieces_contours.append(shifted_contour)
                    board[i][j] = f"{piece_type.capitalize()}"
                else:
                    pass
            else:
                # If no piece then the board is empty there.
                board[i][j] = "."
                
    return board

def capture_board(left, right,top, bottom, square_size, moveList):
    ImageGrab.grab(bbox = (left, top, right, bottom)).save("afterBoard.png")

    # Find the board states of the two active board pictures, before and after
    board1 = board_state("beforeBoard.png",square_size)
    board2 = board_state("afterBoard.png",square_size)
    
    # Depending on which side you are playing the board will be flipped so this accounts for that
    if white == True:
        numbers = ["8","7","6","5","4","3","2","1"]
        letters = ['a','b','c','d','e','f','g','h']
    else:
        numbers = ["1","2","3","4","5","6","7","8"]
        letters = ['h','g','f','e','d','c','b','a']
    
    start_square = ""
    end_square  = ""

    for i in range(8):
        for j in range(8):
            # Comparing each square between the two boards to find the move made
            if board1[i][j] != board2[i][j]:
                
                if board2[i][j] == ".":
                    start_square = f"{letters[j]}{numbers[i]}"
                else:
                    end_square = f"{letters[j]}{numbers[i]}"
            
            else:

                pass
    
    # If there was a move then start and end square will not be "" so it will print and save the move information
    if start_square != "" and end_square != "":
        print(f"{start_square}{end_square}")
        moveList.append(start_square+end_square)
    else:
        pass
    
    # Cleaning up files and reseting for next picture
    os.remove("beforeBoard.png")
    os.rename("afterBoard.png", "beforeBoard.png")
    return


In [52]:
## Initial setup for program, takes the screenshot of the whole screen to find the board location, then takes the screen shot of the starting board to be compared to later.
# This should happend on a starting position board, before moves have been made 

ImageGrab.grab().save("testScreenshot.png")

testim = cv.imread("testScreenshot.png")
left, right, top, bottom = board_finder(testim)
board_size = right - left
square_size = int(board_size/8)

os.remove("testScreenshot.png")

ImageGrab.grab(bbox = (left, top, right, bottom)).save("beforeBoard.png")
im = cv.imread("beforeBoard.png")


In [53]:
## Uses the beforeBoard to find the countours of only six specific locations one for each unique piece save that information in the control_contours list

## Read an image of a the target chessboard in base position
im2 = cv.imread("beforeBoard.png")

## This section reads each square of the board and save the countours of each unique piece in a list 
## to check future contours against and determine piece type
target_squares = [[1,0,"Pawn"],[0,1,"Knight"],[0,2,"Bishop"],[0,0,"Rook"],[0,3,"Queen"],[0,4,"King"]]
control_contours = []

for square in target_squares:
    i = square[1]
    j = square[0]
    piece = square[2]
    
    x = square_size*j #123
    y = square_size*i #123

    square = im2[x:x+square_size , y:y+square_size]

    # Convert the image to grayscaleq
    gray = cv.cvtColor(square, cv.COLOR_BGR2GRAY)

    # Apply GaussianBlur to reduce image noise and improve contour detection
    blurred = cv.GaussianBlur(gray, (3, 3), 0)

    # Use adaptive thresholding to create a binary image
    thresh = cv.adaptiveThreshold(blurred, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY_INV, 7, 1)

    # Find contours in the thresholded image
    contours, _ = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    min_contour_area = 100  # Minimum contour area to be considered a piece
    max_contour_area = 10000

    for cnt in contours:
        if min_contour_area <  cv.contourArea(cnt) < max_contour_area:
            control_contours.append(cnt)
            piece_color = square[int(0.8*square_size), int(0.5*square_size)]
        else:
            pass

if piece_color[0] == 87:
    white = True
else:
    white = False
            
    

In [54]:
## This cell contains the main loop, that runs capture_board every 0.4 seconds which gets the new board state, compares it to the last one and determines the move made. Saves the move data to a csv file.

moveList = []

while True:
    capture_board(left, right, top, bottom, square_size, moveList)
    time.sleep(0.3)
    with open('moveList.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        for move in moveList:
            writer.writerow([move])
    if keyboard.is_pressed('q'): 
        print("Exiting loop.")
        os.remove("beforeBoard.png")
        break



d2d4
d7d5
b1c3
b8c6
e2e3
c8f5
b2b3
e7e5
f1d3
Exiting loop.
